In [8]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Merge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return numpy.array(dataX), numpy.array(dataY)

corMat = pandas.read_csv('correlationmatrix.csv', engine='python')
corMat = corMat.iloc[:,1:21]

dataframe = pandas.read_csv('시뮬레이션 데이터.csv', engine='python')
dataframe = dataframe.iloc[0:46,1:21]
error_scores2= pandas.DataFrame(index=range(20),columns=["5~10년","5~15년","5~20년","5~25년","5~30년","5~35년"])

for i in range(20):
	k=0
	dataframe2 = dataframe.iloc[:,i]
	dataset = dataframe2.values
	dataset = dataset.astype('float32')
	scaler = MinMaxScaler(feature_range=(0, 1))
	dataset = scaler.fit_transform(dataset)
	
	datasetcompare1 = (dataframe.iloc[:,corMat.iloc[i,1]]).values
	datasetcompare1 = datasetcompare1.astype('float32')
	scalercompare1 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare1 = scalercompare1.fit_transform(datasetcompare1)
	

	repeats = 3
	
	for j in [5,10,15,20,25,30]:
		error_scores = list()
		for r in range(repeats):
			numpy.random.seed(random.randrange(1,1000))
			train_size = j
			test_size = len(dataset) - train_size
			train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
			look_back = 1
			trainX, trainY = create_dataset(train, look_back)
			testX, testY = create_dataset(test, look_back)
			trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
			testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
			batch_size = 1
			first_model = Sequential()
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			first_model.add(Dense(1))
			
			
			traincompare1, testcompare1 = datasetcompare1[0:train_size], datasetcompare1[train_size:len(dataset)]
			look_back = 1
			trainXcompare1, trainYcompare1 = create_dataset(traincompare1, look_back)
			testXcompare1, testYcompare1 = create_dataset(testcompare1, look_back)
			trainXcompare1 = numpy.reshape(trainXcompare1, (trainXcompare1.shape[0], 1, trainXcompare1.shape[1]))
			testXcompare1 = numpy.reshape(testXcompare1, (testXcompare1.shape[0], 1, testXcompare1.shape[1]))
			
			second_model = Sequential()
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			second_model.add(Dense(1))
			
			
			model = Sequential()
			model.add(Merge([first_model, second_model], mode='concat'))
			model.add(Dense(1))

			model.compile(loss='mean_squared_error', optimizer='adam')
			for z in range(10):
				model.fit([trainX,trainXcompare1], trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
				model.reset_states()
			trainPredict = model.predict([trainX,trainXcompare1], batch_size=batch_size)
			model.reset_states()
			testPredict = model.predict([testX,testXcompare1], batch_size=batch_size)
			trainPredict = scaler.inverse_transform(trainPredict)
			trainY = scaler.inverse_transform([trainY])
			testPredict = scaler.inverse_transform(testPredict)
			testY = scaler.inverse_transform([testY])
			
			
			testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
			error_scores.append(testScore)
		
		results = pandas.DataFrame()
		results['rmse'] = error_scores
		error_scores2.iloc[i,k]=results.describe().iloc[1,0]
		k=k+1


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 60s - loss: 0.0145
Epoch 1/1
 - 0s - loss: 0.0103
Epoch 1/1
 - 0s - loss: 0.0064
Epoch 1/1
 - 0s - loss: 0.0035
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0014
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 0s - loss: 0.0030
Epoch 1/1
 - 62s - loss: 0.0153
Epoch 1/1
 - 0s - loss: 0.0130
Epoch 1/1
 - 0s - loss: 0.0103
Epoch 1/1
 - 0s - loss: 0.0079
Epoch 1/1
 - 0s - loss: 0.0059
Epoch 1/1
 - 0s - loss: 0.0043
Epoch 1/1
 - 0s - loss: 0.0031
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 59s - loss: 0.0150
Epoch 1/1
 - 0s - loss: 0.0121
Epoch 1/1
 - 0s - loss: 0.0090
Epoch 1/1
 - 0s - loss: 0.0064
Epoch 1/1
 - 0s - loss: 0.0043
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 59s - loss: 0.0496
Epoch 1/1
 - 0s - loss: 0.0361
Epoc

/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 61s - loss: 0.0175
Epoch 1/1
 - 0s - loss: 0.0156
Epoch 1/1
 - 0s - loss: 0.0134
Epoch 1/1
 - 0s - loss: 0.0114
Epoch 1/1
 - 0s - loss: 0.0094
Epoch 1/1
 - 0s - loss: 0.0077
Epoch 1/1
 - 0s - loss: 0.0062
Epoch 1/1
 - 0s - loss: 0.0048
Epoch 1/1
 - 0s - loss: 0.0037
Epoch 1/1
 - 0s - loss: 0.0029
Epoch 1/1
 - 60s - loss: 0.0170
Epoch 1/1
 - 0s - loss: 0.0141
Epoch 1/1
 - 0s - loss: 0.0108
Epoch 1/1
 - 0s - loss: 0.0077
Epoch 1/1
 - 0s - loss: 0.0052
Epoch 1/1
 - 0s - loss: 0.0034
Epoch 1/1
 - 0s - loss: 0.0022
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 60s - loss: 0.0171
Epoch 1/1
 - 0s - loss: 0.0142
Epoch 1/1
 - 0s - loss: 0.0111
Epoch 1/1
 - 0s - loss: 0.0082
Epoch 1/1
 - 0s - loss: 0.0057
Epoch 1/1
 - 0s - loss: 0.0038
Epoch 1/1
 - 0s - loss: 0.0026
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 61s - loss: 0.0618
Epoch 1/1
 - 0s - loss: 0.0458
Epoc

/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

...

Epoch 1/1
 - 132s - loss: 0.0171
Epoch 1/1
 - 0s - loss: 0.0148
Epoch 1/1
 - 0s - loss: 0.0125
Epoch 1/1
 - 0s - loss: 0.0104
Epoch 1/1
 - 0s - loss: 0.0084
Epoch 1/1
 - 0s - loss: 0.0067
Epoch 1/1
 - 0s - loss: 0.0052
Epoch 1/1
 - 0s - loss: 0.0041
Epoch 1/1
 - 0s - loss: 0.0031
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 158s - loss: 0.0174
Epoch 1/1
 - 0s - loss: 0.0151
Epoch 1/1
 - 0s - loss: 0.0122
Epoch 1/1
 - 0s - loss: 0.0094
Epoch 1/1
 - 0s - loss: 0.0069
Epoch 1/1
 - 0s - loss: 0.0049
Epoch 1/1
 - 0s - loss: 0.0034
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 134s - loss: 0.0171
Epoch 1/1
 - 0s - loss: 0.0145
Epoch 1/1
 - 0s - loss: 0.0115
Epoch 1/1
 - 0s - loss: 0.0084
Epoch 1/1
 - 0s - loss: 0.0055
Epoch 1/1
 - 0s - loss: 0.0033
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 132s - loss: 0.0425
Epoch 1/1
 - 0s - loss: 0.0137


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 142s - loss: 0.0160
Epoch 1/1
 - 0s - loss: 0.0139
Epoch 1/1
 - 0s - loss: 0.0119
Epoch 1/1
 - 0s - loss: 0.0099
Epoch 1/1
 - 0s - loss: 0.0081
Epoch 1/1
 - 0s - loss: 0.0066
Epoch 1/1
 - 0s - loss: 0.0052
Epoch 1/1
 - 0s - loss: 0.0041
Epoch 1/1
 - 0s - loss: 0.0032
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 146s - loss: 0.0161
Epoch 1/1
 - 0s - loss: 0.0141
Epoch 1/1
 - 0s - loss: 0.0118
Epoch 1/1
 - 0s - loss: 0.0095
Epoch 1/1
 - 0s - loss: 0.0073
Epoch 1/1
 - 0s - loss: 0.0054
Epoch 1/1
 - 0s - loss: 0.0039
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 0s - loss: 0.0022
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 129s - loss: 0.0156
Epoch 1/1
 - 0s - loss: 0.0109
Epoch 1/1
 - 0s - loss: 0.0058
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0013
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0041
Epoch 1/1
 - 0s - loss: 0.0056
Epoch 1/1
 - 0s - loss: 0.0058
Epoch 1/1
 - 0s - loss: 0.0049
Epoch 1/1
 - 136s - loss: 0.0642
Epoch 1/1
 - 0s - loss: 0.0554


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 157s - loss: 0.0180
Epoch 1/1
 - 0s - loss: 0.0162
Epoch 1/1
 - 0s - loss: 0.0136
Epoch 1/1
 - 0s - loss: 0.0105
Epoch 1/1
 - 0s - loss: 0.0075
Epoch 1/1
 - 0s - loss: 0.0050
Epoch 1/1
 - 0s - loss: 0.0031
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 148s - loss: 0.0172
Epoch 1/1
 - 0s - loss: 0.0131
Epoch 1/1
 - 0s - loss: 0.0088
Epoch 1/1
 - 0s - loss: 0.0053
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0026
Epoch 1/1
 - 0s - loss: 0.0037
Epoch 1/1
 - 0s - loss: 0.0045
Epoch 1/1
 - 162s - loss: 0.0182
Epoch 1/1
 - 0s - loss: 0.0167
Epoch 1/1
 - 0s - loss: 0.0147
Epoch 1/1
 - 0s - loss: 0.0124
Epoch 1/1
 - 0s - loss: 0.0101
Epoch 1/1
 - 0s - loss: 0.0080
Epoch 1/1
 - 0s - loss: 0.0061
Epoch 1/1
 - 0s - loss: 0.0046
Epoch 1/1
 - 0s - loss: 0.0035
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 156s - loss: 0.0410
Epoch 1/1
 - 0s - loss: 0.0086


In [9]:
error_scores2

,5~10년,5~15년,5~20년,5~25년,5~30년,5~35년
0,1.72909,1.13172,1.2511,0.650341,0.924603,0.773811
1,7.43144,7.09891,5.87567,2.9344,2.76775,1.90269
2,14.8434,17.7887,14.0806,12.0762,7.46943,9.84915
3,4.36667,3.17462,2.60594,2.57421,1.42805,1.08001
4,1.09575,0.982883,1.00885,0.646832,0.736605,0.33047
5,13.6211,9.46334,9.67349,8.17936,5.28913,3.4115
6,7.07912,5.11446,4.58327,3.0744,2.61583,2.58133
7,1.34764,1.15712,0.994757,1.04815,0.644069,0.744918
8,0.693668,0.491477,0.485552,0.368528,0.121206,0.269278
9,5.80265,6.19185,5.01538,4.01246,2.60692,3.22291


In [10]:
pandas.DataFrame((error_scores2.mean(axis=0))).iloc[:,0]

5~10년    7.521736
5~15년    6.552873
5~20년    5.407669
5~25년    3.957176
5~30년    3.159870
5~35년    2.608665
Name: 0, dtype: float64